In [1]:
!apt-get update
!apt-get install -y libgl1-mesa-glx

Get:1 http://packages.cloud.google.com/apt gcsfuse-focal InRelease [1227 B]
Get:2 https://packages.cloud.google.com/apt cloud-sdk InRelease [1618 B]       
Get:3 https://packages.cloud.google.com/apt google-fast-socket InRelease [1071 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1581 B]
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]      
Err:1 http://packages.cloud.google.com/apt gcsfuse-focal InRelease          
  The following signatures couldn't be verified because the public key is not available: NO_PUBKEY C0BA5CE6DC6315A3
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease   
Err:2 https://packages.cloud.google.com/apt cloud-sdk InRelease
  The following signatures couldn't be verified because the public key is not available: NO_PUBKEY C0BA5CE6DC6315A3
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Err:3 https://packages.cloud.google.com/apt google-fast-socket InRelease
  The

In [2]:
!pip install deepface
!pip install plotly
!pip install -U scikit-learn
!pip install seaborn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 20.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 62.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.2 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114261 sha256=7fa50c47d1491c97d773d60fc310e6f6b92eee778dfd0ef383f7c3a79de913fd
  Stored in directory: /root/.cache/pip/wheels/3f/10/bd/e267fde82704057e3b60de87ee0ad877baee969d3bb50bced8
Successfully built fire
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall

In [3]:
import cv2
import os
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import math
from PIL import Image
from tqdm import tqdm
import seaborn as sns
from deepface import DeepFace

from keras.models import Model
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.offline as offline
from plotly.subplots import make_subplots
import random

from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical


from tensorflow import keras
from tensorflow.keras.layers import *
from keras.models import *
from keras.applications import *
from keras.optimizers import *
from keras import layers
from keras.regularizers import *
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.vgg19 import VGG19
from keras import regularizers
from sklearn.metrics import confusion_matrix,classification_report
import plotly.graph_objs as go
from sklearn.metrics import  roc_auc_score
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import LabelBinarizer



from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, GlobalAveragePooling2D, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from deepface import DeepFace
import matplotlib.cm as cm
plt.style.use('fivethirtyeight')


25-01-24 08:59:15 - Directory /root/.deepface has been created
25-01-24 08:59:15 - Directory /root/.deepface/weights has been created


In [4]:

class Align_Face():
    def __init__(self):
        #definning opencv path for cascades
        opencv_home = cv2.__file__
        folders = opencv_home.split(os.path.sep)[0:-1]
        path = folders[0]
        for folder in folders[1:]:
            path = path + "/" + folder

        path_for_face = path+"/data/haarcascade_frontalface_default.xml"
        path_for_eyes = path+"/data/haarcascade_eye.xml"
        #loading Cascades
        self.face_detection_cascade = cv2.CascadeClassifier(path_for_face)
        self.eye_detection_cascade = cv2.CascadeClassifier(path_for_eyes)

    def face_detection(self,img):
        #getting bounding box of face
        faces = self.face_detection_cascade.detectMultiScale(img, 1.5, 5)
        if (len(faces) <= 0):
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            return img, img_gray
        else:
            X, Y, W, H = faces[0]
            #getting Face ROI
            img = img[int(Y):int(Y+H), int(X):int(X+W)]
            return img, cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)


    def trignometry_for_distance(self,a, b):
        return math.sqrt(((b[0] - a[0]) * (b[0] - a[0])) +\
                ((b[1] - a[1]) * (b[1] - a[1])))

    # Find eyes
    def Face_Alignment(self,img_path):
        img_raw = cv2.imread(img_path).copy()
        img, gray_img = self.face_detection(cv2.imread(img_path))
        #detecting Eyes
        eyes = self.eye_detection_cascade.detectMultiScale(gray_img)

        #checking if both eyes are present in the image
        if len(eyes) >= 2:
            eye = eyes[:, 2]
            container1 = []
            for i in range(0, len(eye)):
                container = (eye[i], i)
                container1.append(container)
            df = pd.DataFrame(container1, columns=[
                  "length", "idx"]).sort_values(by=['length'])
            eyes = eyes[df.idx.values[0:2]]
            

            eye_1 = eyes[0]
            eye_2 = eyes[1]
            #selecting left and right eye on the basis of their position
            if eye_1[0] > eye_2[0]:
                left_eye = eye_2
                right_eye = eye_1
            else:
                left_eye = eye_1
                right_eye = eye_2


            right_eye_center = (
            int(right_eye[0] + (right_eye[2]/2)),
            int(right_eye[1] + (right_eye[3]/2)))
            right_eye_x = right_eye_center[0]
            right_eye_y = right_eye_center[1]
            cv2.circle(img, right_eye_center, 2, (255, 0, 0), 3)

            left_eye_center = (int(left_eye[0] + (left_eye[2] / 2)),int(left_eye[1] + (left_eye[3] / 2)))
            left_eye_x = left_eye_center[0]
            left_eye_y = left_eye_center[1]
            cv2.circle(img, left_eye_center, 2, (255, 0, 0), 3)


            if left_eye_y > right_eye_y:
#                 print("Rotate image to clock direction")
                point_3rd = (right_eye_x, left_eye_y)
                direction = -1 
            else:
#                 print("Rotate to inverse clock direction")
                point_3rd = (left_eye_x, right_eye_y)
                direction = 1 # rotate inverse direction of clock

            cv2.circle(img, point_3rd, 2, (255, 0, 0), 2)
            #calculating trignometric distance
            a = self.trignometry_for_distance(left_eye_center,
                        point_3rd)
            b = self.trignometry_for_distance(right_eye_center,
                        point_3rd)
            c = self.trignometry_for_distance(right_eye_center,
                        left_eye_center)
            cos_a = (b*b + c*c - a*a)/(2*b*c)
            angle = (np.arccos(cos_a) * 180) / math.pi
            
            #checking if we have to rotate image clock wise or anti clock wise
            if direction == -1:
                angle = 90 - angle
            else:
                angle = -(90-angle)

            new_img = Image.fromarray(img_raw)
            new_img = np.array(new_img.rotate(direction * angle))

        return new_img

In [7]:
class Load_Datasets():
    '''This class takes these parameters as input
    path :path is a required parameter to locate your dataset which have to load if their is train test path is alredy defined in your dataset directory feedit till there
    else give path where the class folders are availaible
    size_images  : with this parameter you can choose height and width of images
    alignFace:if you want to align Face you can keep it True but remember it can ruin some images which are already aligned,we keep this paremeter off by default because our dataset images are already aligned.
    minimum_images: if you want to give minimum images per class should have you can define here otherwise it will take minimum length of class by default and keep all clases with same length
    thresh_minimum: if you want to restrict some any which have less images than this parameter so it will skip that class
    '''
    def __init__(self,path="/kaggle/input/combined-affectnet",size_images= (96,96),gray=False,align_face = False,minimum_images = None,thresh_minimum = None):
        self.path = path
        self.gray = gray
        self.size_images = size_images
        self.align_face = align_face
        self.al_face = Align_Face()
        self.minimum_num_classes = minimum_images
        self.thresh_minimum = thresh_minimum
        self.flags = []
    def load_images(self,p):
        images = []
        labels = []
        #loading each emotion
        for emotion in tqdm(os.listdir(p)):
#             print("="*100)
            print(emotion,": Total Images Present in Dataset: "+str(len(os.listdir(os.path.join(p,emotion)))) )
#             print("="*100)
            #checking if thresh minimum directed any emotion in black list(i mean any particluar class have lesss images than defined in thresh minimum it will be skip)
            if emotion in self.flags:
                print(f"we are skipping this model {emotion} beacause total images present in this class: "+str(len(os.listdir(os.path.join(p,emotion))))+f" is less then thresh_minimum: {thresh_minimum}" )
                continue
            #initialize parameter for legth of images of particular emotion
            k = 0
            new_p = os.path.join(p,emotion)
            for image in os.listdir(new_p):
                #checking if we are less than minimum_images_paramer to stop loading further images
                if k<=self.minimum_num_classes:
                
                    try:
                        #checking if align face argument is true or not
                        if self.align_face:
                            alignedFace = self.al_face.Face_Alignment(os.path.join(new_p,image))
                            aligned_face, gray_aligned_face  = self.al_face.face_detection(alignedFace)
                            if self.gray:
                                image = gray_aligned_face
                            else:
                                image = aligned_face[:, :, ::-1]
                        else:
                            
                            image = cv2.imread(os.path.join(new_p,image))
                            image = image[:,:,::-1]
#                             print(image.shape)
                    except:
#                         print("Error in this image ",emotion,image)
#                         print(image)
                        continue
                    image = cv2.resize(image,self.size_images)
                    images.append(image)
                    labels.append(emotion)
                    k+=1
                else:
                    #break loop if have reached the same length of minimum_image
                    break
        if self.gray:
            images = np.array(images)
            labels = np.array(labels)
            images = images.reshape(images.shape[0],images.shape[1]*images.shape[2])
            images = images.astype("float")/255
            
        return images,labels
    def load_data(self):
        for file in os.listdir(self.path):
            #if there is a train and test directory
            if file.lower()=="train":
                
                print("*"*100)
                print("Loading Training Dataset")
                self.minimum_num_classes = None
                train_path = os.path.join(self.path,"train")
                #checking the class which have less number of images
                for i in os.listdir(train_path):
                    if  self.minimum_num_classes:
                        self.minimum_num_classes = min(len(os.listdir(os.path.join(train_path,i))),self.minimum_num_classes)
                    else:
                         self.minimum_num_classes = len(os.listdir(os.path.join(train_path,i) ) )
                    if self.thresh_minimum != None:
                        if len(os.listdir(os.path.join(train_path,i)))<self.thresh_minimum:
                            self.flags.append(i)
                train_x,train_y = self.load_images(train_path)
            else:
                if file.lower()=="test":
                    print("*"*100)
                    print("Loading Testing Dataset")
                    self.minimum_num_classes = None
                    test_path = os.path.join(self.path,"test")
                     #checking the class which have less number of images
                    for i in os.listdir(test_path):
                        if  self.minimum_num_classes:
                            self.minimum_num_classes = min(len(os.listdir(os.path.join(test_path,i))),self.minimum_num_classes)
                        else:
                             self.minimum_num_classes = len(os.listdir(os.path.join(test_path,i) ) )
                        if self.thresh_minimum != None:
                            if len(os.listdir(os.path.join(test_path,i)))<self.thresh_minimum:
                                self.flags.append(i)
                       
                    test_x,test_y = self.load_images(test_path)
                #if there is no train test directory so directly load dataset with emotions
                else:
                    print("*"*100)
                    print("Loading Dataset Directly | No Train-Test Folders")
                    #checking the class which have less number of images
                    for i in os.listdir(self.path):
                        if  self.minimum_num_classes:
                            self.minimum_num_classes = min(len(os.listdir(os.path.join(self.path,i))),self.minimum_num_classes)
                        else:
                             self.minimum_num_classes = len(os.listdir(os.path.join(self.path,i) ) )
                        if self.thresh_minimum != None:
                            if len(os.listdir(os.path.join(self.path,i)))<self.thresh_minimum:
                                self.flags.append(i)
                    x,y = self.load_images(self.path)
                    return x,y,None,None
        return train_x,train_y,test_x,test_y

In [8]:
data = Load_Datasets(size_images = (224, 224),minimum_images=50)
x,y,_,_= data.load_data()

# data = Load_Datasets(minimum_images=1000,size_images = (96, 96),align_face=False)
# openfacex,openfacey,test_x,test_y = data.load_data()


# data = Load_Datasets(minimum_images=1000,size_images = (55, 47),align_face=False)
# deepidx,deepidy,test_x,test_y = data.load_data()


# data = Load_Datasets(minimum_images=1000,size_images = (152, 152,),align_face=False)
# deepfacex,deepfacey,test_x,test_y = data.load_data()


# data = Load_Datasets(minimum_images=1000,size_images = (160, 160),align_face=False)
# facenetx,facenety,test_x,test_y = data.load_data()


****************************************************************************************************
Loading Dataset Directly | No Train-Test Folders


100%|██████████| 1/1 [00:00<00:00, 137.95it/s]

affectnet-cleaned : Total Images Present in Dataset: 2


In [9]:
df= pd.DataFrame()

df["labels"] =y

In [10]:
def plot_data_distribution(data,label):
    fig = make_subplots(
        rows=1, cols=2,
        column_widths=[0.5, 0.5],
        row_heights=[0.5],
        specs=[[ {"type": "pie"}, {"type": "Funnelarea"}]])
    #plotting Pie Plot

    fig.add_trace(go.Pie(
        labels=data.index, 
        values=data["labels"],
        legendgroup="group",
        textinfo='percent+label'), 
        row=1, col=1)
    #Plotting Funnel Area
    fig.add_trace(go.Funnelarea(
       values=data['labels'], labels=data.index, name='Emotions data distribution',
        title = {"position": "top center",}), 
                        row=1, col=2)

    fig.update_layout(height=500,width=1000, bargap=0.2,
                      margin=dict(b=0,r=20,l=20), xaxis=dict(tickmode='linear'),
                      title_text=f"{label} Data Distribution",
                      template="plotly_white",
                      title_font=dict(size=29, color='#8a8d93', family="Lato, sans-serif"),
                      font=dict(color='#8a8d93'), 
                      hoverlabel=dict(bgcolor="#f2f2f2", font_size=13, font_family="Lato, sans-serif"),
                      showlegend=False)
    fig.show()
data_dist = pd.DataFrame(df["labels"].value_counts() )
plot_data_distribution(data_dist,"AffectNet")

In [11]:
#This function is build forgetting random sample each time for each class
def get_random_unique_sample(images,labels):
    l = []
    imgs = []
    labels_array = np.array(labels)
    #unique labels
    uniq = list(np.unique(labels_array))
    #determining images and adding into list until we get the our unique images for each emotion 
    while len(l)!=len(uniq):
        r = random.randint(0,len(images)-1)
        unknown = labels[r]
        if unknown not in l:
            l.append(labels[r])
            imgs.append(images[r])
    del labels_array
    del uniq
    return  imgs,l
def show_images(images, rows = 1, titles = None,main_title = None):
    assert((titles is None) or (len(images) == len(titles)))
    #for potting Images
    n_images = len(images)
    if titles is None: titles = ['Image (%d)' % i for i in range(1,n_images + 1)]
    fig = plt.figure()
    if main_title:
        fig.suptitle(main_title,fontsize = 35)
    for n, (image, title) in enumerate(zip(images, titles)):
        a = fig.add_subplot(int(rows),int( np.ceil(n_images/float(rows))), n +1)
        if image.ndim == 2:
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_images)

    plt.subplots_adjust(wspace=0, hspace=0)
    fig.tight_layout()
    plt.show()
imgss,labelss = get_random_unique_sample(x,y)
imgss,labelss = np.array(imgss),np.array(labelss)
show_images(imgss, rows = 2, titles = labelss,main_title = "AffectNet Dataset")


<Figure size 0x0 with 0 Axes>

In [12]:
def preprocess(x,y,name):
    print("*"*20,name,"*"*20)
    x,y = np.array(x),np.array(y)
    unique_labels = list(np.unique(y))
    encoded = []
    # craeting encoded labels into numeric number 0:angry etc 
    etol = {k:v for k,v in enumerate(unique_labels)}
    #encoding each label 
    for label in y:
        encoded.append(unique_labels.index(label))
#     encoder = LabelEncoder()
#     y = encoder.fit_transform(y)
    #one hot encoding of each label from 0 -->0000 0000, 1-->0000 0001,2--> 0000 0010 etc
    y = to_categorical(np.array(encoded))
    #splitting training and testing dataset 
    train_x,test_x,train_y,test_y  =train_test_split(x,y,test_size=0.2,shuffle=True)
    print(len(train_x),len(train_y),len(test_x),len(test_y))
    print("ShapeS Train:",train_x.shape,train_y.shape)
    print("ShapeS Test:",test_x.shape,test_y.shape)

    return train_x,test_x,train_y,test_y,etol
train_x,test_x,train_y,test_y,etol =preprocess(x,y,"Data")
# open_train_x,open_test_x,open_train_y,open_test_y =preprocess(openfacex,openfacey,"OpenFace")
# deepface_train_x,deepface_test_x,deepface_train_y,deepface_test_y =preprocess(deepfacex,deepfacey,"DeepFace")
# facenet_train_x,facenet_test_x,facenet_train_y,facenet_test_y =preprocess(facenetx,facenety,"Facenet")
# deepid_train_x,deepid_test_x,deepid_train_y,deepid_test_y =preprocess(deepidx,deepidy,"DeepID")

******************** Data ********************


ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
class Model_Building:
    def __init__(self,model_name=None,train_x=None,train_y=None,test_x=None,test_y=None,model_check_point = "/kaggle/working/"):
        # Constructor for the Model_Building class
        # Initializes the class attributes
        print("-"*20,"Model Name: " ,model_name.upper(),"-"*20)
        self.model_name = model_name
        self.check_point = model_check_point
        # Sets the base model name
        if self.model_name == "vgg19":
            self.preprocess_input = keras.applications.vgg19.preprocess_input
            self.model = VGG19
        elif self.model_name == "xception":
            self.preprocess_input = keras.applications.xception.preprocess_input
            self.model = Xception
        elif self.model_name=="mobilenetv2":
            self.preprocess_input = keras.applications.mobilenet_v2.preprocess_input
            self.model = MobileNetV2
        elif self.model_name == "efficient":
            self.preprocess_input = tf.keras.applications.efficientnet.preprocess_input
            self.model = EfficientNetB7
        else:
            self.preprocess_input = keras.applications.inception_v3.preprocess_input
            self.model = InceptionV3
        # Sets the preprocessing function and the base model based on the model name provided
        self.train_x = train_x
        self.train_y = train_y
        self.test_x = test_x
        self.test_y = test_y
        # Sets the training and testing data
        self.main_model = None
        # Initializes the main model to None
    
    def base_model(self,data,pr_model  ):
        # Builds the base model by extracting the features from the pre-trained model
        w,h = data.shape[1:3]
        # Extracts the width and height from the shape of the data
        cnn_model = pr_model(include_top=False, input_shape=(w, h, 3), weights='imagenet')
        # Initializes the pre-trained model without the fully connected layers
        
        inputs = Input((w, h, 3))
        x = inputs
        x = Lambda(self.preprocess_input, name='preprocessing')(x)
        # Applies the preprocessing function to the input data
        x = cnn_model(x)
        # Extracts the features from the pre-trained model
        x = GlobalAveragePooling2D()(x)
        # Performs global average pooling to reduce the number of parameters
        cnn_model = Model(inputs, x)
        # Sets the base model as the input and the extracted features
        features = cnn_model.predict(data, batch_size=5, verbose=1)
        # Extracts the features from the training or testing data
        return features
    
        # Define a function for the bottom model of the neural network
    def bottom_model(self,features):
        # Determine the percentage of data to be used for training and rest for validation

        # Define the neural network architecture
        model = Sequential()
        model.add(layers.Dense(300, activation="relu", input_shape=(features.shape[1],)))
        model.add(layers.Dense(200, activation="relu"))
        model.add(BatchNormalization())
#         model.add(layers.Dense(1500, activation="relu"))
        model.add(layers.Dense(100, activation="relu", kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(0.2))
        model.add(layers.Dense(len(np.unique(train_y, axis=0)), activation="softmax"))

        # Compile the model with appropriate settings
        model.compile(optimizer=Adam(0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

        # Train the model on the specified data
        

        # Return the trained model and its history
        return model


    # Define a function for running the entire neural network model
    def forward(self, batch_size=4, epochs=10, train_percent=0.90, verbose=1):
        # Extract features using the base model
        f = self.base_model(self.train_x,self.model)
        percent = int(len(f) * train_percent)

        path1 = os.path.join(self.check_point,self.model_name)
        # Train the bottom model using the extracted features
        if not os.path.exists(path1):
            os.mkdir(path1)
        cp_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(path1,"cp-{epoch:04d}.ckpt"), 
        verbose=verbose, 
        save_weights_only=True,
        save_freq=5*batch_size)
        self.main_model = self.bottom_model(f)
        self.main_model.save_weights(self.check_point.format(epoch=0))
        hist = self.main_model.fit(f[:percent], train_y[:percent], validation_data=(f[percent:], train_y[percent:]),
                batch_size=batch_size, epochs=epochs, callbacks=[cp_callback])
#         model.save(f"/kaggle/working/{self.model_name}.h5")
        
        # Return the history of the trained model and the trained model itself
        return hist, self.main_model


    # Define a function for making predictions with the trained neural network model
    def predict(self, sample_test=None):
        if sample_test is None:
            # Extract features using the base model for the test data
            test_f = self.base_model(self.test_x,self.model)

            # Make predictions using the trained model
            p = self.main_model.predict(test_f)
            p = np.argmax(p, axis=1)
        else:
            # Extract features using the base model for the given input data
            test_f = self.base_model( sample_test,self.model)

            # Make predictions using the trained model
            p = self.main_model.predict(test_f)
            p = np.argmax(p, axis=1)

        # Return the predicted classes
        return p


    # Define a function for evaluating the performance of the trained neural network model
    def evaluate(self):
        # Extract features using the base model for the test data
        test_f = self.base_model(self.test_x,self.model)

        # Evaluate the performance of the trained model on the test data
        loss, acc = self.main_model.evaluate(test_f, self.test_y)

        # Print the results
        print("="*50, self.model_name, "="*50)
        print("On Test Data")
        print("Loss: ", round(loss, 4))
        print("Accuracy: ", round(acc, 4))

        # Return the loss and accuracy of the trained model on the test data
        return loss, acc

    

In [ ]:
# #Experiment Trying to check if testing accuracy increase with concatnation of two Models(Function API)
# def get_features(pr_model, data,width=224):
#     #base model
#     cnn_model = pr_model(include_top=False, input_shape=(width, width, 3), weights='imagenet')
    
#     inputs = Input((width, width, 3))
#     x = inputs
#     x = Lambda(preprocess_input, name='preprocessing')(x)
#     x = cnn_model(x)
#     x = GlobalAveragePooling2D()(x)
#     cnn_model = Model(inputs, x)
#     features = cnn_model.predict(data, batch_size=5, verbose=1)
#     return features
# inception_features = get_features(InceptionV3, train_x)
# xception_features = get_features(Xception,train_x)
# #concating two pretrained models features extracted from training data 
# features = np.concatenate([inception_features, xception_features],axis=1)

# model = Sequential()
# model.add(layers.Dense(200,activation= "relu",input_shape= (features.shape[1],)))
# # model.add(layers.Dense(900,activation = "relu"))
# model.add(layers.Dense(100,activation="relu"))
# model.add(Dropout(0.5))
# #model.add(layers.Dense(700,activation="relu"))
# model.add(layers.Dense(8,activation="softmax"))
# model.compile(optimizer = Adam(0.0001) ,loss = "categorical_crossentropy" , metrics = ["accuracy"])
# train_percent = 0.90
# percent = int(len(features)*train_percent)
# history = model.fit(features[:percent],train_y[:percent],validation_data = (features[percent:],train_y[percent:]),batch_size = 5,epochs = 10)

# inception_features = get_features(InceptionV3, test_x)
# xception_features = get_features(Xception,test_x)
# features = np.concatenate([inception_features, xception_features],axis=1)
# eva = model.evaluate(features,test_y)
# print(eva[0],eva[1])

In [ ]:
def plot_Accuracy_Loss(history,name):
    plt.style.use("fivethirtyeight")
    plt.figure(figsize = (20,6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'])
    plt.title(f" {name} Model Loss",fontsize=20)
    plt.ylabel('Loss')
    plt.xlabel('Epochs')
    plt.legend(['train loss', 'validation loss'], loc ='best')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'])
    plt.title(f"{name} Model Accuracy ",fontsize=20)
    plt.ylabel('Accuracy')
    plt.xlabel('Epochs')
    plt.legend(['training accuracy', 'validation accuracy'], loc ='best')
    plt.show()

def Plot_ROC(y_pred,y_test,etol = {0: 'anger', 1: 'contempt', 2: 'disgust', 3: 'fear', 4: 'happy', 5: 'neutral', 6: 'sad', 7: 'surprise'}):
    
    """
    This function takes in predicted and original labels, performs multiclass ROC analysis and
    plots a ROC curve using Plotly.
    
    Parameters:
    y_pred (array-like): Predicted labels
    y_test (array-like): True labels
    
    Returns:
    None
    """
    
    # Convert labels to binary format
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    
    # Compute ROC curve and ROC area for each class
    n_classes = y_test.shape[1]
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        
    # Plot ROC curve for each class using Plotly
    fig = go.Figure()
    for i in range(n_classes):
        fig.add_trace(
            go.Scatter(x=fpr[i], y=tpr[i],
                       name=f'ROC curve (class {list(etol.values())[i]})',
                       line=dict(width=2))
        )
        
    # Add diagonal line for reference
    fig.add_trace(
        go.Scatter(x=[0, 1], y=[0, 1],
                   name='Diagonal line',
                   line=dict(color='gray', dash='dash'))
    )
    
    # Set plot title and axes labels
    fig.update_layout(
        title='Multiclass ROC Curve',
        xaxis=dict(title='False Positive Rate'),
        yaxis=dict(title='True Positive Rate')
    )
    
    # Show plot
    fig.show()

def plot_ROC(y_scores,y_onehot,etol = {0: 'anger', 1: 'contempt', 2: 'disgust', 3: 'fear', 4: 'happy', 5: 'neutral', 6: 'sad', 7: 'surprise'}):
    fig = go.Figure()
    fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0=0, x1=1, y0=0, y1=1
    )

    for i in range(y_scores.shape[1]):
        y_true = y_onehot[:, i]
        y_score = y_scores[:, i]

        fpr, tpr, _ = roc_curve(y_true, y_score)
        auc_score = roc_auc_score(y_true, y_score)

        name = f"{list(etol.values())[i]} (AUC={auc_score:.2f})"
        fig.add_trace(go.Scatter(x=fpr, y=tpr,name=name, mode='lines'))

    fig.update_layout(
        xaxis_title='False Positive Rate',
        yaxis_title='True Positive Rate',
        yaxis=dict(scaleanchor="x", scaleratio=1),
        xaxis=dict(constrain='domain'),
        width=700, height=500
    )
    fig.show()

In [ ]:
#defineing object of each pretrained model
incept = Model_Building("inception",train_x,train_y,test_x,test_y )
incept_hist,incept_model = incept.forward(verbose = 1,batch_size = 16,epochs =70)
incept_loss,incept_acc = incept.evaluate()

xcept = Model_Building("xception",train_x,train_y,test_x,test_y )
xcept_hist,xcept_model = xcept.forward(verbose = 1,batch_size = 16,epochs =70)
xcept_loss,xcept_acc = xcept.evaluate()

mobilenet = Model_Building("mobilenetv2",train_x,train_y,test_x,test_y )
mobilenet_hist,mobilenet_model = mobilenet.forward(verbose = 1,batch_size = 16,epochs =70)
mobilenet_loss,mobilenet_acc = mobilenet.evaluate()

vgg = Model_Building("vgg19",train_x,train_y,test_x,test_y )
vgg_hist,vgg_model = vgg.forward(verbose = 1,batch_size = 16,epochs =70)
vgg_loss,vgg_acc = vgg.evaluate()

efficient = Model_Building("efficient",train_x,train_y,test_x,test_y )
efficient_hist,efficient_model = efficient.forward(verbose = 1,batch_size = 16,epochs =70)
efficient_loss,efficient_acc = efficient.evaluate()

In [ ]:
plot_Accuracy_Loss(incept_hist,"Inception")
plot_Accuracy_Loss(xcept_hist,"Xception")
plot_Accuracy_Loss(mobilenet_hist,"MobileNet")
plot_Accuracy_Loss(vgg_hist,"VGG")
plot_Accuracy_Loss(efficient_hist,"Efficient")

In [ ]:
def predict(model_obj,d={0: 'anger', 1: 'contempt', 2: 'disgust', 3: 'fear', 4: 'happy', 5: 'neutral', 6: 'sad', 7: 'surprise'},test_x=None,size = (224,224),model_path = None,save_img = None):
    '''if you are predicting from saved model , 
    saved model should have their name directory in which models chekpoints are present'''
    print("Predicted Emotions")
    #if there is no text_x(test images array) given so it will take it defined model attribute
    if model_path is None:
        #predict the test images from define model
        results = model_obj.predict()
        l = []
        #inversing the encoded predicted labels form dictinary we defined above
        for i in results:
            l.append(d[i])
        #just showing the sample of each class of predicted output
        imgs,la = get_random_unique_sample(model_obj.test_x,l)
        show_images(imgs,titles=la,rows=2,main_title = "Predicted Emotions by "+model_obj.model_name)
        
        orginal_labels = []
        #inversing the one hot encoded labels of orginal labels
        or_label = np.argmax(model_obj.test_y,axis=1)
        #inversing numeric original labels into actual name 
        for i in or_label:
            orginal_labels.append(d[i])
        #returning test_x,predicted labes and original labels
        return model_obj.test_x,l, orginal_labels
    else:
        #if only one image path is given in parameter
        print("Predicting from Images Paths")
        if type(test_x)==str:
            img = cv2.imread(test_x)
            imgs = cv2.resize(img,size)
            imgs = np.expand_dims(imgs,axis= 0)
            print(imgs.shape)
            
        else:
            #array of images is given in parameter
            imgs = []
            for img in test_x:
                img = cv2.imread(img)
                imgs.append(cv2.resize(img,size))
            imgs = np.array(imgs)
        model_obj = Model_Building(model_name = model_obj)
        path1 = os.path.join(model_path,model_obj.model_name,"cp-{epoch:04d}.ckpt")
        latest = tf.train.latest_checkpoint(os.path.dirname(path1))
        test_f = model_obj.base_model( imgs,model_obj.model)
        model  = model_obj.bottom_model(test_f)
        # Load the previously saved weights
        print(latest)
        model.load_weights(latest)
        
        results = model.predict(test_f)
        results = np.argmax(results, axis=1)
        final_img_emotions = []
        l = []
        for i in results:
            l.append(d[i])
        for pos,img in enumerate(imgs) :
            img = cv2.putText(img, f'Predicted Emotion: {l[pos]}', (5,15),cv2.FONT_HERSHEY_SIMPLEX, 
                   0.5, (0,0,0), 1, cv2.LINE_AA)
            if save_img:
                cv2.imwrite(save_img+str(results[pos])+f"_{pos}.jpg",img[:,:,::-1])
            final_img_emotions.append(img)
       
        if len(l)==1:
            plt.imshow(final_img_emotions[0])
            plt.title(l[0])
            
        else:
            
            imgss,la = get_random_unique_sample(final_img_emotions,l)
            
            #just showing the sample of each class of predicted output

            show_images(imgss,titles=la,rows=2,main_title = "Predicted Emotions ")
        
#         orginal_labels = []
#         or_label = np.argmax(test_y,axis=-1)
#         for i in or_label:
#             orginal_labels.append(d[i])
        return final_img_emotions, l 
#predicting from images paths loading save models
pred_image,pred_l = predict(model_obj = "inception",d = etol,test_x = ["/kaggle/input/young-affectnet-hq/surprise/10_ffhq_101.png","/kaggle/input/young-affectnet-hq/neutral/10_ffhq_1.png"],model_path = "/kaggle/working/",save_img= "/kaggle/working/")

In [ ]:
vgg_testx,vgg_pred,vgg_testy = predict(vgg,etol)
Plot_ROC(vgg_pred,vgg_testy)
print ('\n*\t\tClassification Report VGG:\n', classification_report(vgg_testy,vgg_pred) )

In [ ]:
xcept_testx,xcept_pred,xcept_testy = predict(xcept,etol)
Plot_ROC(xcept_pred,xcept_testy)
print ('\n*\t\tClassification Report Xception:\n', classification_report(xcept_testy,xcept_pred) )

In [ ]:
mobilenet_testx,mobilenet_pred,mobilenet_testy= predict(mobilenet,etol)
Plot_ROC(mobilenet_pred,mobilenet_testy)
print ('\n*\t\tClassification Report MobileNet:\n', classification_report(mobilenet_testy,mobilenet_pred) )

In [ ]:
efficient_testx,efficient_pred,efficient_testy = predict(efficient,etol)
Plot_ROC(efficient_pred,efficient_testy)
print ('\n*\t\tClassification Report Efficient:\n', classification_report(efficient_testy,efficient_pred) )

In [ ]:
incept_testx,incept_pred,incept_testy = predict(incept,etol)
Plot_ROC(incept_pred,incept_testy)
print ('\n*\t\tClassification Report Incption:\n', classification_report(incept_testy,incept_pred) )

In [ ]:
def plot_confusion_matrix(cm, names, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.figure(figsize = (20,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(names))
    plt.xticks(tick_marks, names, rotation=90)
    plt.yticks(tick_marks, names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
def plot_CM(pred_y,original_y,name = None):
    if name is not None:
        print("*"*30,name.upper(),"*"*30)
    cm = confusion_matrix(original_y, pred_y)
    np.set_printoptions(precision=2)
#     print(cm)
    plt.rcParams["figure.figsize"]=(8,5)
    plt.figure()
    if name:
        plot_confusion_matrix(cm,list(etol.values()),title = name)
    else:
         plot_confusion_matrix(cm,list(etol.values()))

def plot_CM( pred_y,original_y,title="Confusion Matrix",categories = etol):
    plt.figure(figsize = (20,10))
    p = random.randint(0,1)
    k = [plt.cm.Greens,plt.cm.Blues]
    ax= plt.subplot()
    cf_matrix = confusion_matrix(original_y, pred_y)
    sns.heatmap(cf_matrix, annot=True,ax=ax,cmap = k[p])
    ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
    ax.set_title(title); 
    ax.xaxis.set_ticklabels(list(etol.values())); ax.yaxis.set_ticklabels(list(etol.values()));
plot_CM(efficient_pred,efficient_testy,"Efficient")


In [ ]:
plot_CM(mobilenet_pred,mobilenet_testy,"Mobilenet")


In [ ]:
plot_CM(incept_pred,incept_testy,"Inception")

In [ ]:
plot_CM(xcept_pred,xcept_testy,"Xception")


In [ ]:
plot_CM(vgg_pred,vgg_testy,"VGG19")

In [ ]:

class GradCAM:
    def __init__(self, model_name="xception", img_size=(299, 299)):
        self.img_size = img_size
        self.preprocess_input = None
        self.decode_predictions = None
        self.last_conv_layer_name = None
        self.model_builder = None
        self.model = None
        
        if model_name == "inception":
            self.preprocess_input = keras.applications.inception_v3.preprocess_input
            self.decode_predictions = keras.applications.inception_v3.decode_predictions
            self.last_conv_layer_name = "mixed10"
            self.model_builder = keras.applications.inception_v3.InceptionV3
        elif model_name == "mobilenetv2":#224,224
            self.preprocess_input = keras.applications.mobilenet_v2.preprocess_input
            self.decode_predictions = keras.applications.mobilenet_v2.decode_predictions
            self.last_conv_layer_name = "Conv_1"
            self.model_builder = keras.applications.mobilenet_v2.MobileNetV2
        elif model_name == "efficient":#600,600
            self.preprocess_input = tf.keras.applications.efficientnet.preprocess_input
            self.decode_predictions = tf.keras.applications.efficientnet.decode_predictions
            self.last_conv_layer_name = "top_conv"
            self.model_builder = tf.keras.applications.EfficientNetB7
        elif model_name == "vgg19":#299,299
            self.preprocess_input = keras.applications.vgg19.preprocess_input
            self.decode_predictions = keras.applications.vgg19.decode_predictions
            self.last_conv_layer_name = "block5_conv4"
            self.model_builder = keras.applications.vgg19.VGG19
        else:#299,299
            self.preprocess_input = keras.applications.xception.preprocess_input
            self.decode_predictions = keras.applications.xception.decode_predictions
            self.last_conv_layer_name = "block14_sepconv2_act"
            self.model_builder = keras.applications.xception.Xception

    def get_img_array(self, img):
        array = cv2.resize(img, self.img_size)
        array = np.expand_dims(array, axis=0)
        return array

    def make_gradcam_heatmap(self, img_array, pred_index=None):
        #loading model for gradient
        grad_model = tf.keras.models.Model(
            [self.model.inputs], [self.model.get_layer(self.last_conv_layer_name).output, self.model.output]
        )

      #taking gradient tape from model weights
        with tf.GradientTape() as tape:
            last_conv_layer_output, preds = grad_model(img_array)
            if pred_index is None:
                pred_index = tf.argmax(preds[0])
            class_channel = preds[:, pred_index]

        #extracted Gradients
        grads = tape.gradient(class_channel, last_conv_layer_output)

        
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
        #last pooled layer

        last_conv_layer_output = last_conv_layer_output[0]
        heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
        heatmap = tf.squeeze(heatmap)

        #creating heatmap of the gradients
        heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
        return heatmap.numpy()


    def display_gradcam(self,img, heatmap,  alpha=0.4):



        heatmap = np.uint8(255 * heatmap)     
        jet = cm.get_cmap("jet")

        #coloring the heatmap from jet colors
        jet_colors = jet(np.arange(256))[:, :3]
        jet_heatmap = jet_colors[heatmap]

        jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
        jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
        jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

        
        superimposed_img = jet_heatmap * alpha + img
        superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)
#         plt.imshow(superimposed_img)
        return superimposed_img
    def forward(self,img):
        #calling all above methods
        img_path = img
        img_array = preprocess_input(self.get_img_array(img_path))

        
        self.model = self.model_builder(weights="imagenet")

        
        self.model.layers[-1].activation = None

        preds = self.model.predict(img_array)


        heatmap = self.make_gradcam_heatmap(img_array)
        result = self.display_gradcam(img_path,heatmap)
        return result


In [ ]:
def display_Grad(model_name="xception",path = None):
    
    '''model_name=xception,inception,mobilenetv2,vgg19,efficient'''
    
    #display the gradient map of PRetrained models
    if model_name =="efficient":
        g = GradCAM(model_name=model_name,img_size = (600,600) )
    elif model_name == "mobilenetv2":
        g = GradCAM(model_name=model_name,img_size = (224,224) )
    elif model_name == "vgg19":
        g = GradCAM(model_name=model_name,img_size = (224,224) )
    else:
        g = GradCAM(model_name=model_name)
    
    
    imgs,la = get_random_unique_sample(x,y)
    grad_images  =[]
    for pos,img in enumerate(imgs):
        result = g.forward(img)
        if path is not None:
            cv2.imwrite(path+f"{model_name}_{la[pos]}_{pos}.jpg",np.array(result))
        grad_images.append(np.array(result))
        
    print("*"*80,model_name,"*"*80)
    show_images(grad_images,titles=la,rows = 2,main_title= model_name.upper())
display_Grad(path = "/kaggle/working/")

In [ ]:
#path for saving gradcams 
display_Grad("inception",path = "/kaggle/working/")

In [ ]:
display_Grad("vgg19",path = "/kaggle/working/")

In [ ]:
display_Grad("mobilenetv2",path = "/kaggle/working/")

In [ ]:
display_Grad("efficient",path = "/kaggle/working/")

In [ ]:
data = Load_Datasets(minimum_images=500,size_images = (224, 224))
vggx,vggy,test_x,test_y = data.load_data()

data = Load_Datasets(minimum_images=500,size_images = (96, 96))
openfacex,openfacey,test_x,test_y = data.load_data()


data = Load_Datasets(minimum_images=500,size_images = (47, 55))
deepidx,deepidy,test_x,test_y = data.load_data()


data = Load_Datasets(minimum_images=500,size_images = (152, 152,))
deepfacex,deepfacey,test_x,test_y = data.load_data()


data = Load_Datasets(minimum_images=500,size_images = (160, 160))
facenetx,facenety,test_x,test_y = data.load_data()

vgg_train_x,vgg_test_x,vgg_train_y,vgg_test_y,vgg_etol =preprocess(vggx,vggy,"VGGFace")
open_train_x,open_test_x,open_train_y,open_test_y ,open_etol=preprocess(openfacex,openfacey,"OpenFace")
deepface_train_x,deepface_test_x,deepface_train_y,deepface_test_y,deepface_etol =preprocess(deepfacex,deepfacey,"DeepFace")
facenet_train_x,facenet_test_x,facenet_train_y,facenet_test_y,facenet_etol =preprocess(facenetx,facenety,"Facenet")
deepid_train_x,deepid_test_x,deepid_train_y,deepid_test_y,deppid_etol =preprocess(deepidx,deepidy,"DeepID")

In [ ]:


# class DeepFaceEmotionClassifier:
#     def __init__(self, model_name='vggface', num_classes=None, input_shape=(96,96, 3)):
#         print("Model: ",model_name.upper())
#         self.model_name = model_name
#         self.num_classes = num_classes
        
#         if self.model_name == 'vggface':
#             self.model = DeepFace.build_model('VGG-Face')
#         elif self.model_name == 'deepid':
#             self.model = DeepFace.build_model('DeepID')
#         elif self.model_name == 'deepface':
#             self.model = DeepFace.build_model('DeepFace')
#         elif self.model_name == 'facenet':
#             self.model = DeepFace.build_model('Facenet')
#         elif self.model_name == 'openface':
#             self.model = DeepFace.build_model('OpenFace')
#         else:
#             raise ValueError('Invalid model_name passed - {}'.format(self.model_name))
        
#         for layer in self.model.layers:
#             layer.trainable = False


#         x = self.model.output
#         x = Flatten()(x)
#         x = Dense(100, activation='relu')(x)
#         predictions = Dense(num_classes, activation='softmax', kernel_initializer='random_uniform')(x)

#         self.emotion_classifier = Model(inputs=self.model.input, outputs=predictions)

#         # Freezing pretrained layers
#         self.model.trainable = False

#         optimizer = Adam()
#         self.emotion_classifier = model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
    
#     def fit(self, X_train, y_train, X_val, y_val, batch_size=32, epochs=50):
#         datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)
#         datagen.fit(X_train)
        
#         hist = self.emotion_classifier.fit(X_train, y_train, batch_size=batch_size, validation_data=(X_val, y_val), epochs=epochs)
#         return hist,self.emotion_classifier
    
#     def predict(self, X):
#         p = self.emotion_classifier.predict(X)
#         p = np.argmax(p,axis=1)
#         return p
#     def evaluate(self,X,y):
#         loss,acc = self.emotion_classifier.evaluate(X)
        
#         print("="*50,self.model_name,"="*50)
#         print("On Test Data")
#         print("Loss: ",round(loss,4))
#         print("Accuracy: ",round(acc,4))
#         return loss,acc 



class DeepFaceEmotionClassifier:
    def __init__(self,X_train, y_train,xval,yval, model_name='vggface', num_classes=len(np.unique(df))):
        self.model_name = model_name
        print("*"*30,self.model_name.capitalize(),"*"*30)
        self.num_classes = num_classes
         
        if self.model_name == 'vggface':
            self.model1 = DeepFace.build_model('VGG-Face')
        elif self.model_name == 'deepid':
            self.model1 = DeepFace.build_model('DeepID')
        elif self.model_name == 'deepface':
            self.model1 = DeepFace.build_model('DeepFace')
        elif self.model_name == 'facenet':
            self.model1 = DeepFace.build_model('Facenet')
        elif self.model_name == 'openface':
            self.model1 = DeepFace.build_model('OpenFace')
        else:
            raise ValueError('Invalid model_name passed - {}'.format(self.model_name))
        #FREEZING LAYERS
        for layer in self.model1.layers[:-4]:
            layer.trainable = False

      
        self.X_train = X_train
        self.y_train = y_train
        self.xval = xval
        self.yval = yval
        self.trainfeatures = self.model1.predict(self.X_train)
        self.valfeatures = self.model1.predict(self.xval)
        self.model = None
    def fit(self,  batch_size=32, epochs=10,verbose=1):
        # Extract features from pre-trained model
        model = Sequential()
        model.add(layers.Dense(20,activation= "relu",input_shape= (self.trainfeatures.shape[1],)))

        #model.add(layers.Dense(700,activation="relu"))
        model.add(layers.Dense(len(np.unique(train_y,axis=0)),activation="softmax"))
        model.compile(optimizer = Adam(0.0001),loss = "categorical_crossentropy" , metrics = ["accuracy"])
        history = model.fit(self.trainfeatures,self.y_train,validation_data = (self.valfeatures,self.yval),verbose=verbose,batch_size =  batch_size,epochs = epochs)    
        self.model = model
        return history,model
    def predict(self, X):
        # Extract features and make predictions
        features = self.model1.predict(X)
        result = self.model.predict(features)
        result = np.argmax(result,axis=-1)
        return result
    def evaluate(self, X,y):
        # Extract features and make predictions
        features = self.model1.predict(X)
        loss,acc = self.model.evaluate(features,y)
        print(self.model_name)
        print("On Test Data")
        print("Accuracy: ",acc)
        print("Loss: ",loss)
        return loss,acc

In [ ]:
percent = int(len(vgg_train_x)*0.90)
openface =  DeepFaceEmotionClassifier(open_train_x[:percent],open_train_y[:percent],open_train_x[percent:],open_train_y[percent:],model_name = 'openface',num_classes = 8)
open_hist,open_model = openface.fit(batch_size = 4)
open_loss,open_acc = openface.evaluate(open_test_x[:percent],open_test_y[:percent])

deepid =  DeepFaceEmotionClassifier(deepid_train_x[:percent],deepid_train_y[:percent],deepid_train_x[percent:],deepid_train_y[percent:],model_name = 'deepid',num_classes = 8)
deepid_hist,deepid_model = deepid.fit(batch_size = 4)
deepid_loss,deepid_acc = deepid.evaluate(deepid_test_x[:percent],deepid_test_y[:percent])


deepface =  DeepFaceEmotionClassifier(deepface_train_x[:percent],deepface_train_y[:percent],deepface_train_x[percent:],deepface_train_y[percent:],model_name = 'deepface',num_classes = 8)
deepface_hist,deepface_model = deepface.fit(batch_size = 4)
deepface_loss,deepface_acc = deepface.evaluate(deepface_test_x[:percent],deepface_test_y[:percent])


facenet = DeepFaceEmotionClassifier(facenet_train_x[:percent],facenet_train_y[:percent],facenet_train_x[percent:],facenet_train_y[percent:],model_name = 'facenet',num_classes = 8)
facenet_hist,facenet_model = facenet.fit(batch_size = 4)
facenet_loss,facenet_acc = facenet.evaluate(facenet_test_x[:percent],facenet_test_y[:percent])


vgg = DeepFaceEmotionClassifier(vgg_train_x[:percent],vgg_train_y[:percent],vgg_train_x[percent:],vgg_train_y[percent:],model_name = 'vggface',num_classes = 8)
vgg_hist,vgg_model = vgg.fit(batch_size = 4)
vgg_loss,vgg_acc = vgg.evaluate(vgg_test_x[:percent],vgg_test_y[:percent])